### Spark HW3: Youtube comments analysis
#### Write a Spark program to analyze the text data.

In this notebook, we have a dataset of user comments for youtube videos related to animals or pets. We will attempt to identify cat or dog owners based on these comments, find out the topics important to them, and then identify video creators with the most viewers that are cat or dog owners.

The dataset provided for this coding test are comments for videos related to animals and/or pets. The dataset is 240MB compressed; please download the file using this google drive link:
https://drive.google.com/file/d/1o3DsS3jN_t2Mw3TsV0i7ySRmh9kyYi1a/view?usp=sharing

 The dataset file is comma separated, with a header line defining the field names, listed here:
● creator_name. Name of the YouTube channel creator.
● userid. Integer identifier for the users commenting on the YouTube channels.
● comment. Text of the comments made by the users.

Please use a recent version of PySpark (version 2.2 or higher) to analyze the data. Do not use
any external libraries; just use the native methods from pyspark.sql and pyspark.ml. (Do not
use pyspark.mllib as this has been deprecated.) Keep your code clean and efficient, with
enough documentation so that the grader can easily follow your train of thought. Summarize
the key results from each step. Explain how to execute your code from a command line
interface.

Step 1: Identify Cat And Dog Owners
Find the users who are cat and/or dog owners.

Step 2: Build And Evaluate Classifiers
Build classifiers for the cat and dog owners and measure the performance of the classifiers.

Step 3: Classify All The Users
Apply the cat/dog classifiers to all the users in the dataset. Estimate the fraction of all users
who are cat/dog owners.

Step 4: Extract Insights About Cat And Dog Owners
Find topics important to cat and dog owners.

Step 5: Identify Creators With Cat And Dog Owners In The Audience
Find creators with the most cat and/or dog owners. Find creators with the highest statistically
significant percentages of cat and/or dog owners.

#### 0. Data Exploration and Cleaning

In [4]:
df_clean=spark.read.csv("/FileStore/tables/animals_comments.csv",inferSchema=True,header=True)
df_clean.show(10)

+--------------------+------+-------------------------------------+
 creator_name|userid| comment|
+--------------------+------+-------------------------------------+
 Doug The Pug| 87.0| I shared this to ...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶|
 bulletproof| 530.0| stop saying get e...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...|
 ojatro|1031.0| I wanna see what ...|
 Tingle Triggers|1212.0| Well shit now Im ...|
Hope For Paws - O...|1806.0| when I saw the en...|
Hope For Paws - O...|2036.0| Holy crap. That i...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...|
 Brian Barczyk|2698.0| Call the teddy Larry|
+--------------------+------+-------------------------------------+
only showing top 10 rows

In [5]:
df_clean.count() 

Out[ 2 ]: 5820035

In [6]:
df_clean = df_clean.na.drop(subset=["comment"])
df_clean.count()

Out[ 3 ]: 5818984

In [7]:
df_clean.show()

+--------------------+------+-------------------------------------+-----+
 creator_name|userid| comment|label|
+--------------------+------+-------------------------------------+-----+
 Doug The Pug| 87.0| I shared this to ...| 0|
 Doug The Pug| 87.0| Super cute 😀🐕🐶| 0|
 bulletproof| 530.0| stop saying get e...| 0|
 Meu Zoológico| 670.0| Tenho uma jiboia ...| 0|
 ojatro|1031.0| I wanna see what ...| 0|
 Tingle Triggers|1212.0| Well shit now Im ...| 0|
Hope For Paws - O...|1806.0| when I saw the en...| 0|
Hope For Paws - O...|2036.0| Holy crap. That i...| 0|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...| 0|
 Brian Barczyk|2698.0| Call the teddy Larry| 0|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟| 0|
Hope For Paws - O...|2911.0| That mother cat l...| 0|
Hope For Paws - O...|2911.0| Its people like H...| 0|
 Talking Kitty Cat|2911.0| steve: No wet foo...| 0|
 Brave Wilderness|3224.0| Dont call this a ...| 0|
 MaxluvsMya|3267.0| why are you alway...| 0|
Rise Up Society F...|3372.0| Deb Tucker| 0|
 The Dodo|3466.0| Thats a deer isnt...| 0|
 Brave Wilderness|3466.0| there is no safe ...| 0|
 Brave Wilderness|3466.0| Red before yellow...| 0|
+--------------------+------+-------------------------------------+-----+
only showing top 20 rows

In [8]:
# find user with preference of dog and cat
from pyspark.sql.functions import when
from pyspark.sql.functions import col

# you can user your ways to extract the label

df_clean = df_clean.withColumn("label", \
                           (when(col("comment").like("%my dog%"), 1) \
                           .when(col("comment").like("%I have a dog%"), 1) \
                           .when(col("comment").like("%my cat%"), 1) \
                           .when(col("comment").like("%I have a cat%"), 1) \
                           .when(col("comment").like("%my puppy%"), 1) \
                           .when(col("comment").like("%I have a puppy"), 1) \
                           .when(col("comment").like("%my pup%"), 1) \
                           .when(col("comment").like("%I have a pup"), 1) \
                           .when(col("comment").like("%my kitty%"), 1) \
                           .when(col("comment").like("%I have a kitty"), 1) \
                           .when(col("comment").like("%my pussy%"), 1) \
                           .when(col("comment").like("%I have a pussy"), 1) \
                           .otherwise(0)))

In [9]:
# check the amount of label 1 in data set
df_clean.filter(df_clean.label == 1).count()

Out[ 6 ]: 40250

#### 1. Data preprocessing and Build the classifier

In [11]:
from pyspark.ml.feature import RegexTokenizer, Word2Vec
from pyspark.ml.classification import LogisticRegression

# regular expression tokenizer
regexTokenizer = RegexTokenizer(inputCol="comment", outputCol="words", pattern="\\W")

word2Vec = Word2Vec(inputCol="words", outputCol="features")

In [12]:
from pyspark.ml import Pipeline

pipeline = Pipeline(stages=[regexTokenizer, word2Vec])

# Fit the pipeline to training documents.
pipelineFit = pipeline.fit(df_clean)
dataset = pipelineFit.transform(df_clean)

In [13]:
dataset.show()

+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
 creator_name|userid| comment|label| words| features|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
 Doug The Pug| 87.0| I shared this to ...| 0|[i, shared, this,...|[0.16463485749607...|
 Doug The Pug| 87.0| Super cute 😀🐕🐶| 0| [super, cute]|[-0.2862297892570...|
 bulletproof| 530.0| stop saying get e...| 0|[stop, saying, ge...|[0.08130848780274...|
 Meu Zoológico| 670.0| Tenho uma jiboia ...| 0|[tenho, uma, jibo...|[0.02678274425367...|
 ojatro|1031.0| I wanna see what ...| 0|[i, wanna, see, w...|[0.09030846811153...|
 Tingle Triggers|1212.0| Well shit now Im ...| 0|[well, shit, now,...|[-0.0707143530249...|
Hope For Paws - O...|1806.0| when I saw the en...| 0|[when, i, saw, th...|[0.02964533811375...|
Hope For Paws - O...|2036.0| Holy crap. That i...| 0|[holy, crap, that...|[0.07307457064206...|
 Life Story|2637.0|武器はクエストで貰えるんじゃないん...| 0| []| (100,[],[])|
 Brian Barczyk|2698.0| Call the teddy Larry| 0|[call, the, teddy...|[0.08389361947774...|
 The Dodo|2702.0| 😐🤔😓😢😭😭😭😭😟| 0| []| (100,[],[])|
Hope For Paws - O...|2911.0| That mother cat l...| 0|[that, mother, ca...|[0.03807020248760...|
Hope For Paws - O...|2911.0| Its people like H...| 0|[its, people, lik...|[0.02404675868650...|
 Talking Kitty Cat|2911.0| steve: No wet foo...| 0|[steve, no, wet, ...|[-0.0292669534683...|
 Brave Wilderness|3224.0| Dont call this a ...| 0|[dont, call, this...|[0.08125947828271...|
 MaxluvsMya|3267.0| why are you alway...| 0|[why, are, you, a...|[0.01138780568726...|
Rise Up Society F...|3372.0| Deb Tucker| 0| [deb, tucker]|[-0.0381043888628...|
 The Dodo|3466.0| Thats a deer isnt...| 0|[thats, a, deer, ...|[0.06501139029860...|
 Brave Wilderness|3466.0| there is no safe ...| 0|[there, is, no, s...|[-0.0106814446694...|
 Brave Wilderness|3466.0| Red before yellow...| 0|[red, before, yel...|[0.03709157285629...|
+--------------------+------+-------------------------------------+-----+--------------------+--------------------+
only showing top 20 rows

In [14]:
(lable0_train,lable0_test)=dataset.filter(col('label')==1).randomSplit([0.7, 0.3],seed = 100)
(lable1_train, lable1_ex)=dataset.filter(col('label')==0).randomSplit([0.005, 0.995],seed = 100)
(lable1_test, lable1_ex2)=lable1_ex.randomSplit([0.002, 0.998],seed = 100)

In [15]:
trainingData = lable0_train.union(lable1_train)
testData=lable0_test.union(lable1_test)
trainingData.show()

+-------------------+---------+--------------------+-----+--------------------+--------------------+
 creator_name| userid| comment|label| words| features|
+-------------------+---------+--------------------+-----+--------------------+--------------------+
 null|1509707.0|It seems some of ...| 1|[it, seems, some,...|[0.04320649328181...|
 null|1931756.0|I have seen my ca...| 1|[i, have, seen, m...|[-0.0220514512139...|
76Highboy Reloading|1558889.0|Actually I use ba...| 1|[actually, i, use...|[0.01596286406463...|
 ASPCA| 8394.0|I never knew ther...| 1|[i, never, knew, ...|[-0.0086244517782...|
 Aarons Animals| 319065.0|This is what my c...| 1|[this, is, what, ...|[0.07929850704967...|
 Aarons Animals| 391041.0|Now my cat wants ...| 1|[now, my, cat, wa...|[-0.0406595712993...|
 Aarons Animals| 926306.0|1:54 Oh shit its ...| 1|[1, 54, oh, shit,...|[-0.0447140565762...|
 Aarons Animals| 937383.0|This is like my c...| 1|[this, is, like, ...|[0.14452461153268...|
 Aarons Animals| 937383.0|when my cat needs...| 1|[when, my, cat, n...|[-0.0110443245619...|
 Aarons Animals|1000140.0|Your cats are cut...| 1|[your, cats, are,...|[0.01555152162909...|
 Aarons Animals|1246398.0|This is the best ...| 1|[this, is, the, b...|[0.10924330419477...|
 Aarons Animals|1538465.0|Yeah my cat does ...| 1|[yeah, my, cat, d...|[0.08626053327073...|
 Aarons Animals|1991739.0|I watch my cats f...| 1|[i, watch, my, ca...|[0.18454209342598...|
 Aarons Animals|2086870.0|Horribly fake. Wa...| 1|[horribly, fake, ...|[-0.0082934946753...|
 Alex Knappenberger| 8896.0|yeah my dog does ...| 1|[yeah, my, dog, d...|[0.09301538299769...|
 Alex Knappenberger| 194047.0|Does Bruce give y...| 1|[does, bruce, giv...|[0.02756243714919...|
 Alex Knappenberger| 194047.0|Hahaha... thats s...| 1|[hahaha, thats, s...|[-0.0027542514846...|
 Alex Knappenberger| 517292.0|Kid thinks he is ...| 1|[kid, thinks, he,...|[-0.0366241580573...|
 Alex Knappenberger| 952189.0|Thats my dog rose...| 1|[thats, my, dog, ...|[0.01302736832035...|
 Alex Knappenberger| 953002.0|It looks like my ...| 1|[it, looks, like,...|[0.04829208061306...|
+-------------------+---------+--------------------+-----+--------------------+--------------------+
only showing top 20 rows

In [16]:
print("Dataset Count: " + str(dataset.count()))
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))

Dataset Count: 5818984
Training Dataset Count: 56923
Test Dataset Count: 23346

##### LogisticRegression

In [18]:
# import packages
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
import time

# define the lr model
lr = LogisticRegression(labelCol="label", featuresCol="features")

##### Parameter Tuning and K-fold cross-validation

In [20]:
start_time = time.time()

# select the best hyperparameters for LR model by useing 5-fold cross validation
paramGrid_lr = ParamGridBuilder() \
    .addGrid(lr.maxIter, [5, 10, 20]) \
    .addGrid(lr.regParam, [0.1, 0.5, 2.0]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

crossval_lr = CrossValidator(estimator=lr, \
                          estimatorParamMaps=paramGrid_lr, \
                          evaluator=BinaryClassificationEvaluator(), \
                          numFolds=5)  

# train the model and get the best model
cvModel_lr = crossval_lr.fit(trainingData)
best_model_lr = cvModel_lr.bestModel
summary = cvModel_lr.bestModel.summary

# get the best hyperparam
best_iter = best_model_lr._java_obj.getMaxIter()
best_reg_param = best_model_lr._java_obj.getRegParam()
best_elasticnet_param = best_model_lr._java_obj.getElasticNetParam()

# get the area under ROC curve of the best model on validation data
eval_lr = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
roc_score = eval_lr.evaluate(summary.predictions)

print ('The hyperparam of the best model: iterations = {}, reg = {}, ElasticNetParam = {}' \
       .format(best_iter, best_reg_param, best_elasticnet_param))
print ('The area under ROC curve of the best model on validation data is:' + str(roc_score))
print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time))

The hyperparam of the best model: iterations = 10, reg = 0.1, ElasticNetParam = 0.0
The area under ROC curve of the best model on validation data is:0.8967835459334892
Total Runtime: 1444.02 seconds

##### RandomForest

In [22]:
# import package
from pyspark.ml.classification import RandomForestClassifier

# define the random forest model
rf = RandomForestClassifier(labelCol="label", featuresCol="features")

##### Parameter Tuning and K-fold cross-validation

In [24]:
start_time2 = time.time()

# select the optimal hyperparams for RF model by using 5-fold validation
paramGrid_rf = ParamGridBuilder() \
    .addGrid(rf.numTrees, [3, 5, 10]) \
    .addGrid(rf.maxDepth, [3, 5, 10]) \
    .build()

crossval_rf = CrossValidator(estimator = rf, \
                            estimatorParamMaps=paramGrid_rf, \
                            evaluator=BinaryClassificationEvaluator(), \
                            numFolds=5)

# train the model and get the best one
cvModel_rf = crossval_rf.fit(trainingData)
best_model_rf = cvModel_rf.bestModel

# get the best hyperparams
best_numTrees = best_model_rf._java_obj.getNumTrees()
best_depth = best_model_rf._java_obj.getMaxDepth()

print ('The hyperparam of the best model: numTrees = {}, maxDepth = {}'.format(best_numTrees, best_depth))
print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time2))

The hyperparam of the best model: numTrees = 10, maxDepth = 10
Total Runtime: 1355.94 seconds

##### Gradient boosting

In [26]:
# import package
from pyspark.ml.classification import GBTClassifier

# define the gradient boosting tree model
gbt = GBTClassifier(labelCol="label", featuresCol="features")

##### Parameter Tuning and K-fold cross-validation

In [28]:
start_time3 = time.time()

# select optimal hyperparams for GBT by using 5-fold cross validation
paramGrid_gbt = ParamGridBuilder() \
    .addGrid(gbt.maxIter, [5, 10]) \
    .addGrid(gbt.maxDepth, [3, 5, 10]) \
    .build()

crossval_gbt = CrossValidator(estimator=gbt, \
                            estimatorParamMaps=paramGrid_gbt, \
                            evaluator=BinaryClassificationEvaluator(), \
                            numFolds=5)

# train the model and get the best one
cvModel_gbt = crossval_gbt.fit(trainingData)
best_model_gbt = cvModel_gbt.bestModel

# get the best hyperparams
best_iter_gbt = best_model_gbt._java_obj.getMaxIter()
best_depth_gbt = best_model_gbt._java_obj.getMaxDepth()

print ('The hyperparam of the best model: iterations = {}, maxDepth = {}'.format(best_iter_gbt, best_depth_gbt))
print ('Total Runtime: {:.2f} seconds'.format(time.time() - start_time3))

The hyperparam of the best model: iterations = 10, maxDepth = 10
Total Runtime: 1753.45 seconds

In [29]:
# make prediction on test data by three best models 
predictions_lr = cvModel_lr.transform(testData)
predictions_rf = cvModel_rf.transform(testData)
predictions_gbt = cvModel_gbt.transform(testData)

# get the area under ROC curve of these three models
roc_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label', metricName='areaUnderROC')
roc_lr = roc_eval.evaluate(predictions_lr)
roc_rf = roc_eval.evaluate(predictions_rf)
roc_gbt = roc_eval.evaluate(predictions_gbt)

print('The roc score of lr is {}, rf is {}, roc_gbt is {}.'.format(roc_lr, roc_rf, roc_gbt))

The roc score of lr is 0.8935662293180607, rf is 0.9018348636383343, roc_gbt is 0.8786764636772458.

1) For logistic regression, the optimal hyper-parameters are iterations = 10, reg = 0.1, ElasticNetParam = 0.0. And the AUC score of it on testing data is 0.8936

2) For random forest, the optimal hyper-parameters are numTrees = 10, maxDepth = 10. And the AUC score of it on testing data is 0.9018

3) For gradient boosting tree, the optimal hyper-parameters are maxIter = 10, maxDepth = 10. And the AUC score of it on testing data is 0.8787

However, I think the performance of RF and GBT can still be improved because the optimal hyper-parameters chosen by the cross-validation are the largest values that given by me. This means the performance of these two models may be improved if the hyper-parameters can be larger.

#### 2. Classify All The Users

In [32]:
# since the performance of random forest is best on test data, choose use it here
predictions_all = cvModel_rf.transform(dataset)
roc_all_rf = roc_eval.evaluate(predictions_all)
print('The AUC score of this model on all model is:' + str(roc_all_rf))

The AUC score of this model on all model is:0.91191580135628

In [33]:
# calculate the fraction of dog and cat owners
from pyspark.sql.functions import countDistinct
amount_owners = predictions_all \
                      .filter(predictions_all.label == 1) \
                      .agg(countDistinct(predictions_all.userid)) \
                      .collect()[0][0]

amount_all = predictions_all \
                      .agg(countDistinct(predictions_all.userid)) \
                      .collect()[0][0]

fraction = amount_owners / amount_all
print('The fraction of dog and cat owners is {:.4f}'.format(fraction))

The fraction of dog and cat owners is 0.0139

Based on the classification result, the fraction of oweners of cat and dogs of all users is about 1.39%. However, this result is not accurate enough because my system only focuses on the English comments.

#### 3. Get insigts of Users

In [36]:
# extract the dog and cat owners' comment to see their topic
owner_comment = predictions_all \
                        .filter(predictions_all.label == 1) \
                        .select(predictions_all.comment)
display(owner_comment)

comment
Now I want to try that with my dog!!!
I blow smoke in my cats ear right to his brain
my dog lucky wont eat of his bowl hell only eat out peoples hands how do i get him to eat out of his bowl
thats what my dog do
Im so happy i think Im almost crying Im hugging my dog Ik its not a cat but its a animal that need love
My cat scratches at it I spray at her but not her so it scars her if she keeps doing it I will spray her ya she stoped for a wile then now she is doing it agin ☹️ ya I always like my door shut and if she is in here in the morning she will want out and Im like IM TRYING TO SLEEEP STOOOOP PLZ IM TIRD 😭😭😭😭then someone will let her out and Im like yaaaaas 5 mins of peace but its hard for me to sleep alone like I have to have my kitty or I get sad and lonely and feel kinda unsafe but she make me feel safe and she keeps me safe and I keep her safe!
Since my cat is getting old Im gonna start calling him by a new name..GRANDPAW!!How is cat food sold?USUALLY PURR CAN!!GIVEAWAY ENTRY!!!!
I have several plants of catnip planted around our garden but my cats dont really seem bothered by it? Are my cats constantly high or something???
This is so sad because my dog died and the mom looks just like her and I started crying
my cat died today im sad woching this video


According to the comments of the cat and dog owners, they are mainly interested in the topics about:

1) The interesting behaviors of cats and dogs

2) The diseases that can contract their pets

3) The friendly act done by human to cats and dogs

#### 4. Identify Creators With Cat And Dog Owners In The Audience

In [39]:
# extract the creators of the dog and cat owners
owner_creator = predictions_all \
                        .filter(predictions_all.label == 1) \
                        .groupby(predictions_all.creator_name) \
                        .count() \
                        .orderBy('count', ascending=False)
display(owner_creator)

creator_name,count
The Dodo,4112
Cole & Marmalade,2904
Gohan The Husky,2354
Zak Georges Dog Training rEvolution,2213
Hope For Paws - Official Rescue Channel,1882
Vet Ranch,1748
Gone to the Snow Dogs,1720
Brian Barczyk,1612
Robin Seplut,1584
Taylor Nicole Dean,1575


Based on the names of creators that attract owners of cats and dogs to comment, we can see that these names are mainly about:

1) Veterinarians, pet training or animal rescue channel

2) Some famous dogs or cats on the internet

3) Some funny videos about cats and dogs

#### 5. Analysis and Future work

##### 1) overview of project 
This project mainly focus on identifying the owners of cats and dogs from all users by their comments. After that, I do some research on the toptics and creators that attract them.

##### 2) data clean and analysis
There are several important points of the data processing in this project. Firstly, I need to define the identification to extract the label from comments. Then, I need to apply word2Vec method here to convert their comments to features. Finally, I need to do the down-sampling on the label 0 because this dataset is very unbalanced and the models I apply are all supervised learning models which are sensitive to unbalanced data.

##### 3) build ml model
For all the models applied in this project, I create cross-validation to find the optimal hyper-parameters for them. And the metric I used in this project is AUC score since this metric can show the performance of models on various aspects.

##### 4) recommendation based on the model results
Based on the result of this project, I advise the creators who want to attract more owners should pay attention to their name. They should make a name that is relative to dogs and cats. What's more, I also find out that owners of dogs and cats are very attractive by the famous pets on the internet.